Initializations:

In [281]:
%reload_ext snakeviz
%matplotlib inline
%run SBRG2.py

#Core
##Pauli Algebra
Pauli matrix class. A Pauli matrix is denoted as
$$\sigma=\mathrm{i}^{x\cdot z}\prod_{i\in Xs}X_i\prod_{i\in Zs}Z_i=\mathrm{i}^{x\cdot z}\prod_{i}X_i^{x_i}Z_i^{z_i},$$
where $x_i=\delta(i\in Xs)$, $z_i=\delta(i\in Zs)$ are vectors of 0,1, and $X_i$, $Z_i$ are the Pauli matrix on site $i$.

Pauli matrices can be created by `mkMat`
- from Xs, Zs sets: `mkMat(Xs, Zs)`,
- or from dict of indices: `mkMat({i:mu, ...})`,
- or from list of indices: `mkMat([m0, mu1, ...])`.

In [252]:
[mkMat({0,1},{1,2}), mkMat({0:1,1:2,2:3}), mkMat([1,2,3])]

[<Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>, <Xs:[0, 1] Zs:[1, 2]>]

`Mat` is hashable and can be compared.

In [248]:
print(hash(mkMat()))
mkMat() == mkMat([])

6795291966596493067


True

The commutativity of two Pauli operators can be checked by `is_commute`

In [249]:
is_commute(mkMat([0,1,0]),mkMat([1,2,3]))

False

Merge two Pauli operators by indices. Coefficients are not calculated here. The coefficient can be restored from the commutation relations later.

In [250]:
pdot(mkMat([0,1,0]),mkMat([1,2,3]))

<Xs:[0] Zs:[1, 2]>

###Pauli Monomial
A term of Pauli operator `Term.mat` with a constant multiple `Term.val`.

In [255]:
[Term(), Term(mkMat([1,2])),Term(mkMat([1,2]),-3.2)]

[1.0 <Xs:[] Zs:[]>, 1.0 <Xs:[0, 1] Zs:[1]>, -3.2 <Xs:[0, 1] Zs:[1]>]

Consider $\sigma_A=\mathrm{i}^{x_A\cdot z_A} X_A Z_A$, $\sigma_B=\mathrm{i}^{x_B\cdot z_B} X_B Z_B$, then the product reads:
$$\sigma_A\sigma_B=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B+2 z_A\cdot x_B}\quad (X_A X_B) (Z_A Z_B)=\mathrm{i}^{x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B}\quad \mathrm{i}^{x_C\cdot z_C}X_C Z_C=\mathrm{i}^{n^{C}_{AB}}\; \sigma_C,$$
where $X_C=X_AX_B$, $Z_C=Z_AZ_B$ and $z_C = (z_A+z_B)\mod 2$. The power of $\mathrm{i}$ is
$$n^{C}_{AB}=x_A\cdot z_A+x_B\cdot z_B-x_C\cdot z_C+2 z_A\cdot x_B$$

In [256]:
dot(Term(mkMat([1,2])),Term(mkMat([3,3])))

1.0 <Xs:[0, 1] Zs:[0]>

In [257]:
idot(Term(mkMat([2,0])),Term(mkMat([3,3])))

-1.0 <Xs:[0] Zs:[1]>

##Hamiltonian
Hamiltonian `Ham` is a sum of Pauli monomial terms. It can be created from a list of terms.

In [258]:
H = Ham(
    [Term(mkMat([0,1,1]),0.5),
     Term(mkMat([3,0,0]),0.2),
     Term(mkMat([0,3,3]),0.8),
     Term(mkMat([1,3,0]),1.0)])
H

[1.0 <Xs:[0] Zs:[1]>, 0.8 <Xs:[] Zs:[1, 2]>, 0.5 <Xs:[1, 2] Zs:[]>, 0.2 <Xs:[] Zs:[0]>]

In [259]:
H.forward([Term(mkMat([0],[0,1]),-1.)])
H

[1.0 <Xs:[] Zs:[0]>, 0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>, -0.2 <Xs:[0] Zs:[1]>]

In [260]:
H.extend([Term(mkMat([1,3,0]),0.1)])
H

[1.0 <Xs:[] Zs:[0]>, 0.8 <Xs:[] Zs:[1, 2]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>, -0.1 <Xs:[0] Zs:[1]>]

In [5]:
H.remove(H.terms[1])
H

[1.0 <Xs:[] Zs:[0]>, -0.1 <Xs:[0] Zs:[1]>, -0.5 <Xs:[0, 1, 2] Zs:[0, 1]>]

In [261]:
print([H.mats[term.mat] == term for term in H])
print([[i in (term.mat.Xs | term.mat.Zs) for term in terms] for (i, terms) in H.imap.items()])
print([term.pos for term in H])

[True, True, True, True]
[[True, True, True], [True, True, True], [True, True]]
[0, 1, 2, 3]


## SBRG

Profiling

In [286]:
system = SBRG(TFIsing(512,J=2.,K=1.,h=1.,alpha=1.))
% snakeviz system.flow()

 
*** Profile stats marshalled to file '/var/folders/tl/lwpcq5qj049ftcj7pvhkzv_h0000gn/T/tmpe9yw_a'. 


Benchmark with SBRG1

In [282]:
random.seed(2)
system = SBRG(TFIsing(8,J=1.,K=1.,h=1.,alpha=1.)).run()
list(system.taus)

[1.598556653186726 <Xs:[0, 7] Zs:[]>,
 1.0907717060778832 <Xs:[] Zs:[2, 3]>,
 1.4535546388675367 <Xs:[] Zs:[5, 6]>,
 0.7384672351453768 <Xs:[] Zs:[6]>,
 0.5735749804876529 <Xs:[] Zs:[4, 5]>,
 0.9335081618920336 <Xs:[] Zs:[1, 2]>,
 0.6766078760265358 <Xs:[] Zs:[3, 4]>,
 -0.4158627935942054 <Xs:[0, 7] Zs:[0, 1, 6, 7]>]

##Measurement

###Level Statistics

In [42]:
random.seed(2)
system = SBRG(TFIsing(14,J=2.,K=1.,h=1.,alpha=1.)).run()
from itertools import chain, combinations
def subsets(iterable):
    s = list(iterable)
    return map(set, chain.from_iterable(combinations(s, r) for r in range(len(s)+1)))
emap = [[s,0] for s in subsets(range(0,system.bits))];
for term in system.Heff:
    Zs = term.mat.Zs
    v = term.val
    for em in emap:
        if len(Zs & em[0])%2 == 1:
            em[1] += v
e0 = sum(term.val for term in system.Heff)
es = [e0-2*em[1] for em in emap]
export('Es_J2K1h1_a10_14',es)

###RG Flow

In [288]:
import math
realization = 64
bins = [x/10 for x in range(-1,301)]
hdic = {}
for i in range(realization):
    print(i)
    system = SBRG(TFIsing(512,J=2.,K=1.,h=1.,alpha=1.))
    n = len(system.phybits)
    while n>=4:
        hs = [-math.log(abs(term.val)) for term in system.H]
        hist, bin_edges = np.histogram(hs,bins)
        try:
            hdic[n] += hist
        except:
            hdic[n] = hist
        system.flow(n/2)
        n = len(system.phybits)
hdic = {n: hist.tolist() for n, hist in hdic.items()}
export('RG_J2K1h1_a10_512', {'bins':bins,'hdic':hdic})

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63


In [88]:
hdic = {n: hist.tolist() for n, hist in hdic.items()}
export('RG_J2K1h1_a10_512', {'bins':bins,'hdic':hdic})

In [280]:
bool([])

False

###Entanglement

In [95]:
class Ent:
    def __init__(self, sys):
        self.bits = sys.bits
        self.phybits = set(range(self.bits))
        self.mat2is = {}
        self.i2mats = {}
        for term in sys.taus.terms:
            mat = term.mat
            sites = mat.Xs | mat.Zs
            self.mat2is[term.mat] = sites
            for i in sites:
                try:
                    self.i2mats[i].add(mat)
                except:
                    self.i2mats[i] = {mat}
        self.region = set()
        self.shared = set()
    def extend(self, newbits):
        for i in newbits:
            self.shared.update(self.i2mats[i])

In [96]:
ent = Ent(system)
ent.extend({0,1,2})
ent.shared

set()

In [79]:
a = set()
a.union({1,2})
a

set()